In [30]:
import torch
import pandas as pd
from torch.utils.data import Dataset , DataLoader
import numpy as np
import torch.nn as nn

In [31]:
torch.__version__

'2.2.1'

In [32]:
ok_gesture=pd.read_csv("./csv/ok.csv")
yes_gesture=pd.read_csv("./csv/yes.csv")
zero_gesture=pd.read_csv("./csv/zero.csv")
call_gesture=pd.read_csv("./csv/call.csv")
five_gesture=pd.read_csv("./csv/five.csv")
one_gesture=pd.read_csv("./csv/one.csv")
rock_gesture=pd.read_csv("./csv/rock.csv")
love_gesture=pd.read_csv("./csv/love.csv")

In [33]:
ok_gesture["gesture"]="ok"
yes_gesture["gesture"]="yes"
zero_gesture["gesture"]="zero"
call_gesture["gesture"]="call"
five_gesture["gesture"]="five"
one_gesture["gesture"]="one"
rock_gesture["gesture"]="rock"
love_gesture["gesture"]="love"


In [34]:
train_dataset=pd.concat([ok_gesture.iloc[0:80,:],call_gesture.iloc[0:80,:],
                         five_gesture.iloc[0:80,:],rock_gesture.iloc[0:80,:],
                         yes_gesture.iloc[0:80,:],zero_gesture.iloc[0:80,:],
                         one_gesture.iloc[0:80,:],love_gesture.iloc[0:80,:]],axis=0)

valid_dataset=pd.concat([ok_gesture.iloc[80:,:],call_gesture.iloc[80:,:],
                         five_gesture.iloc[80:,:],rock_gesture.iloc[80:,:],
                         yes_gesture.iloc[80:,:],zero_gesture.iloc[80:,:],
                         one_gesture.iloc[80:,:],love_gesture.iloc[80:,:]],axis=0)

In [35]:
train_dataset.describe()

,1w,1x,1y,1z,2w,2x,2y,2z,3w,3x,...,4y,4z,5w,5x,5y,5z,6w,6x,6y,6z
count,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,...,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000
mean,0.680868,0.444212,0.194902,-0.083684,0.482260,0.037983,0.065109,-0.016899,0.383770,-0.224236,...,-0.097230,-0.078783,0.540888,-0.427375,-0.068199,0.056125,0.710243,0.595227,-0.060364,0.084543
std,0.222951,0.432057,0.162752,0.178034,0.272622,0.711097,0.289214,0.314679,0.282018,0.747637,...,0.237288,0.301862,0.281561,0.528156,0.268926,0.295386,0.209936,0.224666,0.125496,0.142742
min,0.010733,-0.879133,-0.216999,-0.634326,-0.012170,-0.999522,-0.631354,-0.676226,-0.013249,-0.999944,...,-0.679968,-0.674037,-0.015593,-0.995405,-0.668391,-0.615134,0.021086,-0.915720,-0.457970,-0.401166
25%,0.544554,0.106266,0.083120,-0.197017,0.275814,-0.721162,-0.103989,-0.268406,0.124190,-0.876250,...,-0.193560,-0.301566,0.287158,-0.830162,-0.244417,-0.164357,0.538303,0.454915,-0.147966,-0.011587
50%,0.706579,0.632133,0.168485,-0.091440,0.484394,-0.000322,0.052998,-0.051777,0.342208,-0.640108,...,-0.084560,-0.113783,0.584709,-0.640207,-0.029167,0.074885,0.783920,0.586885,-0.045791,0.034229
75%,0.858487,0.772912,0.286596,0.010543,0.703879,0.797996,0.245369,0.282419,0.629755,0.649235,...,0.007821,0.129517,0.794212,-0.167084,0.110779,0.287433,0.878515,0.787850,0.020157,0.164331
max,0.998440,0.990957,0.660559,0.642290,0.999101,0.999907,0.695531,0.687305,0.991465,0.999261,...,0.578039,0.674976,0.994355,0.997932,0.543820,0.666804,0.998437,0.982811,0.413545,0.563559


In [36]:
gesture_labels=sorted(list(set(train_dataset["gesture"])))
n_classes=len(gesture_labels)
print(n_classes)
print(gesture_labels)

8
['call', 'five', 'love', 'ok', 'one', 'rock', 'yes', 'zero']


In [37]:
class_to_num=dict(zip(gesture_labels,range(n_classes)))

In [38]:
num_to_class={v:k for k,v in class_to_num.items()}

In [39]:
print(class_to_num)
print(num_to_class)

{'call': 0, 'five': 1, 'love': 2, 'ok': 3, 'one': 4, 'rock': 5, 'yes': 6, 'zero': 7}
{0: 'call', 1: 'five', 2: 'love', 3: 'ok', 4: 'one', 5: 'rock', 6: 'yes', 7: 'zero'}


In [40]:
class Gesturedataset(Dataset):

    def __init__(self,data_frame,mode="train") -> None:
        self.reallen=len(data_frame)-1
        
        if mode =="train":
            self.mode="train"
            # tensor geshi shuju label feature fenkai
            self.train_data=torch.tensor(data_frame.iloc[1:,0:-1].values)
            self.train_label=np.asarray(data_frame.iloc[1:,-1].values)

        elif mode == "valid":
            self.mode="valid"
            self.valid_data=torch.tensor(data_frame.iloc[1:,0:-1].values)
            self.valid_label=np.asarray(data_frame.iloc[1:,-1].values)
        print("the {} mode of gesture data. {} samples found".format(mode,self.reallen))

    def __getitem__(self, index) :

        if self.mode == "train":
            quaternion_6part=self.train_data[index]
            label=self.train_label[index]
            number_label=class_to_num[label]
            return quaternion_6part,number_label
        else:
            quaternion_6part=self.valid_data[index]
            label=self.valid_label[index]
            number_label=class_to_num[label]
            return quaternion_6part,number_label

    
    def __len__(self):
        return self.reallen

In [41]:
train_dataset_forload=Gesturedataset(train_dataset,mode="train")
valid_dataset_forload=Gesturedataset(valid_dataset,mode="valid")

the train mode of gesture data. 639 samples found
the valid mode of gesture data. 159 samples found


In [42]:
qua,label=train_dataset_forload[300]
print(qua,label)

tensor([ 0.7646,  0.6035,  0.1848, -0.1306,  0.2201,  0.8770,  0.1138, -0.4117,
         0.3223,  0.6745, -0.2820,  0.6014,  0.7408, -0.0889, -0.6606, -0.0829,
         0.3100, -0.8230,  0.1566,  0.4496,  0.7899,  0.5856,  0.1528, -0.0986],
       dtype=torch.float64) 5


In [43]:
train_loader=DataLoader(train_dataset_forload,batch_size=16,shuffle=True)
valid_loader=DataLoader(valid_dataset_forload,batch_size=16,shuffle=True)

In [44]:
device='cpu'

In [54]:
class MLP(nn.Module):
    def __init__(self) :
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(24,24),nn.ReLU(),
            nn.Linear(24,14),nn.ReLU(),
            nn.Linear(14,8)
        )
        # 初始化模型的权重和偏置为 Float 类型
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                layer.weight.data = layer.weight.data.double()
                layer.bias.data = layer.bias.data.double()
                
    def forward (self,x):
        return self.net(x)

In [56]:
# CHAO CAN SHU
learning_rate = 0.1
weight_decay = 2e-3
num_epoch = 40
model_path = './MLP_model .ckpt'

In [57]:
model=MLP()
model=model.to(device)

# loss and optim
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

total_train_step=0
total_test_step=0

for i in range(num_epoch):
    print("----------{} turn of learning begins---------".format(i))

    train_accuracy=[]
    train_loss=[]
    # training
    model.train()
    for data,label in train_loader:
        output=model(data)
        loss=loss_fn(output,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step+=1
        
        acc=(output.argmax(dim=-1)==label).float().mean()
        train_accuracy.append(acc)
        train_loss.append(loss)
        print("training time {}, loss {}".format(total_train_step,loss))
    
    train_loss=sum(train_loss)/len(train_loss)
    train_accuracy=sum(train_accuracy)/len(train_accuracy)
    # Print the information.
    print(f"[ Train | {i + 1:03d}/{num_epoch:03d} ] loss = {train_loss:.5f}, acc = {train_accuracy:.5f}")

    model.eval()
    valid_loss=[]
    valid_accuracy=[]
    # validation
    for data,label in valid_loader:
        with torch.no_grad():
            output=model(data)

        loss=loss_fn(output,label)
        acc=(output.argmax(dim=-1)==label).float().mean()
        total_test_step+=1
        
        valid_accuracy.append(acc)
        valid_loss.append(loss)
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_accuracy = sum(valid_accuracy) / len(valid_accuracy)
    print(f"[ Valid | {i + 1:03d}/{num_epoch:03d} ] loss = {valid_loss:.5f}, acc = {valid_accuracy:.5f}")


----------0 turn of learning begins---------
training time 1, loss 2.115028008114264
training time 2, loss 2.0209425821215876
training time 3, loss 2.016866648427307
training time 4, loss 2.066486648860237
training time 5, loss 2.122699840947993
training time 6, loss 2.137396580828099
training time 7, loss 1.9852638449581341
training time 8, loss 2.029897834952049
training time 9, loss 1.9766715293629165
training time 10, loss 1.9228164430895778
training time 11, loss 2.001734138250028
training time 12, loss 1.9251122062479404
training time 13, loss 1.7362589416690377
training time 14, loss 1.5940935739497901
training time 15, loss 1.9627740502208353


training time 16, loss 1.4737960390056926
training time 17, loss 1.8359389509600692
training time 18, loss 1.1569822117188089
training time 19, loss 1.8629880236615532
training time 20, loss 1.4436036013528848
training time 21, loss 1.8146290664360314
training time 22, loss 1.7734956075544348
training time 23, loss 1.9159223181121403
training time 24, loss 1.7265220321361405
training time 25, loss 2.1161550381072387
training time 26, loss 1.627162888489273
training time 27, loss 2.1367627964926967
training time 28, loss 1.6079675073147721
training time 29, loss 1.5639843951010457
training time 30, loss 1.5596062994502908
training time 31, loss 1.8809236995952032
training time 32, loss 1.3897141710698506
training time 33, loss 1.3977737645374677
training time 34, loss 1.0842862305159604
training time 35, loss 1.7045836448644112
training time 36, loss 1.4051147410553761
training time 37, loss 1.4006837658828075
training time 38, loss 1.9098082836210089
training time 39, loss 1.3548535586

In [59]:
torch.save(model.state_dict(),model_path)